<a href="https://colab.research.google.com/github/fjadidi2001/Image_Inpaint/blob/main/Image_Inpaint_CelebHQ_Diffusions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install diffusers transformers accelerate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset
from torchvision import transforms
import torch
import numpy as np
import cv2
from diffusers import DDPMPipeline, DDPMScheduler
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import Adam

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [4]:
ds = load_dataset("saitsharipov/CelebA-HQ")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/224 [00:00<?, ?B/s]

(…)-00000-of-00003-30fbdf94a5d8e722.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

(…)-00001-of-00003-74b418aa7a721747.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

(…)-00002-of-00003-4f9459a488896f53.parquet:   0%|          | 0.00/465M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/202599 [00:00<?, ? examples/s]

In [5]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Note: this should be 256x256 to match model's expected input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [6]:
def preprocess(examples):
    examples['image'] = [transform(image) for image in examples['image']]
    return examples

ds = ds.map(preprocess, batched=True)

Map:   0%|          | 0/202599 [00:00<?, ? examples/s]

In [15]:
def create_irregular_mask(image_size=(128, 128), max_vertices=8, max_brush_width=30):
    mask = np.ones(image_size, dtype=np.uint8)
    num_shapes = np.random.randint(1, 5)
    for _ in range(num_shapes):
        vertices = np.random.randint(0, max(image_size), size=(np.random.randint(3, max_vertices), 2))
        cv2.fillPoly(mask, [vertices], color=0)
        for _ in range(np.random.randint(1, 3)):
            pt1 = tuple(np.random.randint(0, max(image_size), size=2))
            pt2 = tuple(np.random.randint(0, max(image_size), size=2))
            cv2.line(mask, pt1, pt2, color=0, thickness=np.random.randint(1, max_brush_width))
    return mask

In [8]:
model = DDPMPipeline.from_pretrained("google/ddpm-celebahq-256")
scheduler = DDPMScheduler.from_pretrained("google/ddpm-celebahq-256")

model_index.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/455M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--google--ddpm-celebahq-256/snapshots/cd5c944777ea2668051904ead6cc120739b86c4d: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--google--ddpm-celebahq-256/snapshots/cd5c944777ea2668051904ead6cc120739b86c4d.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


In [9]:
class InpaintingModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, noisy_images, masks, timesteps):
        model_input = torch.cat([noisy_images, masks], dim=1)
        return self.model(model_input, timesteps).sample

inpainting_model = InpaintingModel(model.unet)

In [10]:
train_dataloader = DataLoader(ds['train'], batch_size=8, shuffle=True)

In [11]:
optimizer = Adam(inpainting_model.parameters(), lr=1e-4)

In [17]:
num_epochs = 10
for epoch in range(num_epochs):
    for batch in train_dataloader:
        # Ensure images are a tensor
        images = batch['image']
        if isinstance(images, list):
            images = torch.stack([torch.from_numpy(np.array(img)) for img in images])
        elif isinstance(images, torch.Tensor):
            pass
        else:
            raise ValueError("Unexpected type for images")

        images = images.to(torch.float32)

        # Create masks with correct size
        masks = torch.stack([torch.from_numpy(create_irregular_mask()).float().unsqueeze(0) for _ in range(images.size(0))])
        # Ensure masks have the same number of channels as images
        masks = masks.unsqueeze(1)  # Shape now: [batch_size, 1, 1, height, width]

        # Expand masks to match image dimensions
        masks = masks.expand(-1, images.size(1), -1, -1, -1)  # Match image dimensions with 5D mask

        masks = masks.to(images.device)

        masked_images = images * (1 - masks)

        timesteps = torch.randint(0, scheduler.num_train_timesteps, (images.size(0),), device=images.device).long()

        noise = torch.randn_like(images)
        noisy_images = scheduler.add_noise(masked_images, noise, timesteps)

        predicted_noise = inpainting_model(noisy_images, masks, timesteps)

        loss = nn.functional.mse_loss(predicted_noise, noise)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

RuntimeError: The size of tensor a (8) must match the size of tensor b (128) at non-singleton dimension 4

In [ ]:
from datasets import load_dataset
from torchvision import transforms
import torch
import numpy as np
import cv2
from diffusers import DDPMPipeline, DDPMScheduler
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import Adam

# Load the CelebA-HQ dataset
ds = load_dataset("saitsharipov/CelebA-HQ")

# Define a transformation pipeline
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Changed to 256x256 to match model's expected size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Apply transformations to the dataset
def preprocess(examples):
    examples['image'] = [transform(image) for image in examples['image']]
    return examples

ds = ds.map(preprocess, batched=True)

def create_irregular_mask(image_size=(256, 256), max_vertices=8, max_brush_width=30):
    mask = np.ones(image_size, dtype=np.uint8)
    num_shapes = np.random.randint(1, 5)
    for _ in range(num_shapes):
        vertices = np.random.randint(0, max(image_size), size=(np.random.randint(3, max_vertices), 2))
        cv2.fillPoly(mask, [vertices], color=0)
        for _ in range(np.random.randint(1, 3)):
            pt1 = tuple(np.random.randint(0, max(image_size), size=2))
            pt2 = tuple(np.random.randint(0, max(image_size), size=2))
            cv2.line(mask, pt1, pt2, color=0, thickness=np.random.randint(1, max_brush_width))
    return mask

# Load a pre-trained diffusion model
model = DDPMPipeline.from_pretrained("google/ddpm-celebahq-256")
scheduler = DDPMScheduler.from_pretrained("google/ddpm-celebahq-256")

# Modify the model for inpainting
class InpaintingModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    def forward(self, noisy_images, masks, timesteps):
        # Concatenate masks as an additional channel
        model_input = torch.cat([noisy_images, masks], dim=1)
        return self.model(model_input, timesteps).sample

inpainting_model = InpaintingModel(model.unet)

# Create a DataLoader
train_dataloader = DataLoader(ds['train'], batch_size=8, shuffle=True)

# Define optimizer
optimizer = Adam(inpainting_model.parameters(), lr=1e-4)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for batch in train_dataloader:
        # Convert images to tensor if not already
        images = batch['image']
        if isinstance(images, list):
            images = torch.stack([torch.from_numpy(np.array(img)) for img in images])
        elif isinstance(images, torch.Tensor):
            pass
        else:
            raise ValueError("Unexpected type for images")

        images = images.to(torch.float32)

        # Create masks
        masks = torch.stack([torch.from_numpy(create_irregular_mask()).float().unsqueeze(0) for _ in range(images.size(0))])
        masks = masks.unsqueeze(1)  # Add channel dimension

        # Ensure masks match image dimensions
        masks = masks.expand(-1, images.size(1), -1, -1, -1)  # Match image dimensions

        masks = masks.to(images.device)

        # Apply masks to images
        masked_images = images * (1 - masks)

        # Sample random timesteps
        timesteps = torch.randint(0, scheduler.num_train_timesteps, (images.size(0),), device=images.device).long()

        # Add noise to the masked images
        noise = torch.randn_like(images)
        noisy_images = scheduler.add_noise(masked_images, noise, timesteps)

        # Predict the noise
        predicted_noise = inpainting_model(noisy_images, masks, timesteps)

        # Compute loss
        loss = nn.functional.mse_loss(predicted_noise, noise)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

In [3]:
from datasets import load_dataset
from torchvision import transforms
import torch
import numpy as np
import cv2
from diffusers import DDPMPipeline, DDPMScheduler
from torch import nn
from torch.utils.data import DataLoader, default_collate
from torch.optim import Adam
from PIL import Image

# Load the CelebA-HQ dataset
ds = load_dataset("saitsharipov/CelebA-HQ")

# Define transformation pipeline for on-the-fly processing
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Changed to 256x256 to match model's expected size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

def create_irregular_mask(image_size=(256, 256), max_vertices=8, max_brush_width=30):
    mask = np.ones(image_size, dtype=np.uint8)
    num_shapes = np.random.randint(1, 5)
    for _ in range(num_shapes):
        vertices = np.random.randint(0, max(image_size), size=(np.random.randint(3, max_vertices), 2))
        cv2.fillPoly(mask, [vertices], color=0)
        for _ in range(np.random.randint(1, 3)):
            pt1 = tuple(np.random.randint(0, max(image_size), size=2))
            pt2 = tuple(np.random.randint(0, max(image_size), size=2))
            cv2.line(mask, pt1, pt2, color=0, thickness=np.random.randint(1, max_brush_width))
    return mask

# Load a pre-trained diffusion model
model = DDPMPipeline.from_pretrained("google/ddpm-celebahq-256")
scheduler = DDPMScheduler.from_pretrained("google/ddpm-celebahq-256")

# Modify the model for inpainting
class InpaintingModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    def forward(self, noisy_images, masks, timesteps):
        # Concatenate masks as an additional channel
        model_input = torch.cat([noisy_images, masks], dim=1)
        return self.model(model_input, timesteps).sample

inpainting_model = InpaintingModel(model.unet)

# Custom collate function to handle PIL images
def custom_collate(batch):
    return {
        'image': [transform(img.convert('RGB')) for img in batch['image']]
    }

# Create a DataLoader with custom collate_fn
train_dataloader = DataLoader(ds['train'], batch_size=8, shuffle=True, collate_fn=custom_collate)

# Define optimizer
optimizer = Adam(inpainting_model.parameters(), lr=1e-4)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for batch in train_dataloader:
        images = batch['image']
        images = torch.stack(images)  # Now images are already tensors

        # Create masks
        masks = torch.stack([torch.from_numpy(create_irregular_mask()).float().unsqueeze(0) for _ in range(images.size(0))])
        masks = masks.unsqueeze(1)  # Add channel dimension

        # Ensure masks match image dimensions
        masks = masks.expand(-1, images.size(1), -1, -1, -1)  # Match image dimensions

        masks = masks.to(images.device)

        # Apply masks to images
        masked_images = images * (1 - masks)

        # Sample random timesteps
        timesteps = torch.randint(0, scheduler.num_train_timesteps, (images.size(0),), device=images.device).long()

        # Add noise to the masked images
        noise = torch.randn_like(images)
        noisy_images = scheduler.add_noise(masked_images, noise, timesteps)

        # Predict the noise
        predicted_noise = inpainting_model(noisy_images, masks, timesteps)

        # Compute loss
        loss = nn.functional.mse_loss(predicted_noise, noise)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--google--ddpm-celebahq-256/snapshots/cd5c944777ea2668051904ead6cc120739b86c4d: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--google--ddpm-celebahq-256/snapshots/cd5c944777ea2668051904ead6cc120739b86c4d.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


TypeError: list indices must be integers or slices, not str